# Drum Transcriber (Omnizart Edition) Colab

Run this notebook to launch the **SOTA Omnizart-powered** Drum Transcriber Web UI.

**Note**: This notebook installs **Python 3.10** to support Omnizart's dependencies.

In [ ]:
# @title 0. Install Python 3.10 (Required for Omnizart)
# @markdown This cell installs Python 3.10 and sets it as default, because Colab's default Python is too new for Omnizart.

import sys

# Install Python 3.10 and distutils
!sudo apt-get update -y
!sudo apt-get install python3.10 python3.10-distutils python3.10-dev -y

# Change default python to 3.10
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
!sudo update-alternatives --set python3 /usr/bin/python3.10

# Install pip for 3.10
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3.10

# Verify version
!python --version


In [ ]:
# @title 1. Setup Environment
# @markdown This cell installs library dependencies.

import os

# Install System Dependencies
!sudo apt-get install libsndfile1 ffmpeg -y

# Install Python Dependencies (now in Python 3.10)
# upgrade build tools
!pip install --upgrade pip setuptools wheel

# 1. Install Build Deps (Numpy & Cython needed for Madmom build)
!pip install "numpy<2.0" cython

# 2. Install Madmom specifically (ignoring isolation to use installed numpy)
!pip install madmom --no-build-isolation

# 3. Install other deps
!pip install pretty_midi gradio yt-dlp librosa==0.8.1 pandas plotly scikit-learn soundfile

# Install Omnizart from source (to be safe)
if not os.path.exists("omnizart"):
    !git clone https://github.com/Music-and-Culture-Technology-Lab/omnizart.git

%cd omnizart
# Patch Setup.py
# 1. Allow Python 3.10 (Remove <3.9 constraint)
!sed -i 's/python_requires=">=3.6, <3.9"/python_requires=">=3.6"/g' setup.py
!sed -i 's/python_requires=">= 3.6, < 3.9"/python_requires=">= 3.6"/g' setup.py

# 2. Patch Deps
!sed -i 's/PyYAML<6.0.0,>=5.3.1/PyYAML>=5.3.1/' setup.py
!sed -i 's/numba==0.48/numba>=0.48/' setup.py
!sed -i 's/librosa==0.8.0/librosa>=0.8.0/' setup.py

!pip install .
%cd ..

# Clone DrumTranscriber Repo
repo_url = "https://github.com/AgentHitmanFaris/DrumTranscriber.git"
repo_name = "DrumTranscriber"

if os.path.exists(repo_name):
    %cd {repo_name}
    !git pull
else:
    !git clone {repo_url}
    %cd {repo_name}

# Download Omnizart checkpoints via Python
# Note: We need to run this with the `python` command because the notebook kernel might still be old python
!python -c "from omnizart.remote import download_checkpoints; download_checkpoints()"


In [ ]:
# @title 2. Run App (Omnizart Mode)
# @markdown This will launch the Gradio interface using the Omnizart model.

launcher_code = """
import os
import gradio_app
from omnizart_wrapper import OmnizartWrapper

# Patch the load_model function to return OmnizartWrapper
def load_omnizart():
    print('Loading Omnizart Wrapper...')
    return OmnizartWrapper()

gradio_app.load_model = load_omnizart

# Run the demo
gradio_app.demo.launch(share=True)
"""

with open('run_omnizart.py', 'w') as f:
    f.write(launcher_code)

!python run_omnizart.py
